__初始化变量__

In [21]:
csv_filename = r"D:\Onedrive\ACGN相关\切煤代购\夜宵称重表\20240803\国内18带重量.csv"
#赋值变量
ppg = 0.12  #单位克重
inLand_post = 2



# 空变量
empty_num = 0  # 当前累计空行数
text_num = 0  # 当前累计非空行数
cn_group = []  # 记录cn的列表
gram_group = []  # 记录重量的列表（与cn索引对应）
total_gram = 0  # 总重
m_num = None  # m码

#总表列表
fina_cn_group=[]
fina_gram_group=[]
fina_price_group=[]

__处理模块__

In [22]:
def run_csv(row_list):
    global empty_num, text_num, m_num, cn_group, gram_group, total_gram
    # 将csv转换为列表
    row_list = [item for item in row]
    # 判断空行
    if row_list[0] == '' and empty_num != 1:  # 第一个空行
        empty_num = 1
        text_num = 0
        # 计算并写入
        if m_num != '':
            total_price_gram = count_total_postage()
            price_group = count_and_write_csv(*total_price_gram)
        # 列表归零
        # 调试代码
        if gram_group == [''] or gram_group == [' '] or gram_group == [0]:
            gram_group = []
            gram_group.append(total_gram)
        print(f"{m_num},{total_gram}\n{cn_group}\n{gram_group}\n{price_group}")
        count_fina_group(cn_group, gram_group, price_group)

        print("-"*30)  # 调试代码
        cn_group = []
        gram_group = []
        price_group = []
        return
    if row_list[0] == '' and empty_num == 1:  # 连续出现的第二个空行
        empty_num = 2
        return
    # 不是空行
    if row_list[0] != '':
        text_num += 1
        empty_num = 0
        if text_num == 1:
            m_num = row_list[0]
            total_gram = row_list[1]
        if text_num > 1:
            find_and_add_cn(row_list[0], row_list[1])
            return


def count_fina_group(cn_group, gram_group, price_group):
    global fina_cn_group, fina_gram_group, fina_price_group
    for cn in cn_group:
        gram = gram_group.pop(0)
        price = price_group.pop(0)
        try:
            # cn已存在
            cn_index = fina_cn_group.index(cn)
            fina_gram_group[cn_index] += float(gram)
            fina_price_group[cn_index] += float(price)
        except ValueError:
            # cn不存在
            fina_cn_group.append(cn)
            fina_gram_group.append(float(gram))
            fina_price_group.append(float(price))
    return


def count_and_write_csv(t_price, t_gram):  # 计算每一单的国际
    out_price_group = []  # 输出的价格列表，索引与cn一致
    # 计算
    if len(gram_group) == 1:
        # 不是拼盘（重量列表只有一个值）
        out_price_group.append(t_price)
    else:
        # 是拼盘，需要算均摊
        for item in gram_group:
            if item == '':
                item = '0'
            out_price_group.append(float(item)*t_price/t_gram)
    return out_price_group


def count_total_postage():
    # 计算总肾额
    total_gram_int = 0  # 总重
    total_price = 0  # 总肾额
    try:
        total_gram_int = int(total_gram)
    except:
        pass
    # 总重
    try:
        # 如果有总重，则取总重或称重和的较大值
        auto_total_gram = sum(float(item) for item in gram_group)
        try:
            total_gram_int = int(total_gram)
        except ValueError:
            pass
        out_total_gram = max(auto_total_gram, total_gram_int)
    except ValueError:
        # 如果没有总重（总重值转float返回失败），则取称重和
        try:
            total_gram_int = int(total_gram)
        except ValueError:
            return -1
        out_total_gram = total_gram_int

    total_price = out_total_gram*ppg + inLand_post

    return [total_price, out_total_gram]


def find_and_add_cn(cn, gram):  # 建立每单的列表，写入数据
    if gram == '':
        gram = 0
    try:
        # cn存在
        cn_index = cn_group.index(cn)
        gram_group[cn_index] += int(gram)
    except ValueError:
        # cn不存在
        cn_group.append(cn)
        gram_group.append(gram)

__业务模块__

In [23]:
import csv

# 打开CSV文件
with open(csv_filename, newline='', encoding='utf-8-sig') as csvfile:
    # 创建CSV阅读器
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        row_list = [item for item in row]
        run_csv(row_list)
        if empty_num == 2:
            break
        
output_path = csv_filename.replace(".csv", "汇总表.csv")
output_list = [
    f"{cn},{round(float(gram), 2)},{price}" for cn, gram, price in zip(fina_cn_group, fina_gram_group, fina_price_group)
]
with open(output_path,'w', newline='', encoding='utf-8-sig') as csvfile:
    # 创建CSV写入器
    csvwriter = csv.writer(csvfile)
    
    # 将每个元素写入CSV文件的一行
    for row in output_list:
        csvwriter.writerow(row.split(','))

m37765976078,
['紫罗兰']
['46.5']
[7.58]
------------------------------
m44310246674,
['百川']
['54.5']
[8.54]
------------------------------
m68914364647,8
['Yan', 'umi', '森理', '清游']
['2', '2', '2', '2']
[0.74, 0.74, 0.74, 0.74]
------------------------------
m99713419191,23
['风佰', '憂一乘', '川林', '啊晶']
['4.8', '2.3', '2.3', '2.3']
[0.9933913043478261, 0.4759999999999999, 0.4759999999999999, 0.4759999999999999]
------------------------------
m48103511964,
['百川', '法法']
['14.8', '14.8']
[2.776, 2.776]
------------------------------
m14055714309,
['显卡']
['27.5']
[5.3]
------------------------------
